<a href="https://colab.research.google.com/github/nik-hil-24/bazingo/blob/main/init0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Get Dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-07-29 16:21:23--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2023-07-29 16:21:23 (33.1 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
# Imports
import torch
from torch import nn
from torch.optim import Adam
from torch.nn import functional as F

In [3]:
# Read Dataset
with open('input.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()

In [4]:
# Dataset Length
len(text)

1115394

In [5]:
# Unique Characters in the Dataset
characters = list(set(text))
print(''.join(sorted(characters)))
print(len(characters))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# Encode Text
mapping = {char:i for i, char in enumerate(characters)}
rev_mapping = dict(enumerate(characters))

encode_text = lambda string: [mapping[s] for s in string]
decode_text = lambda ls: ''.join([rev_mapping[l] for l in ls])

# Test
print(encode_text('Hi There!'))
print(decode_text([57, 39, 29, 38, 4, 42, 14, 42, 55]))

[5, 43, 24, 29, 57, 62, 12, 62, 61]
h?TUqPfPd


In [7]:
# Creating Tensor Dataset of Encoded Text
data = torch.tensor(encode_text(text), dtype = torch.long)
print(data.shape)

torch.Size([1115394])


In [8]:
# Train Test Split
n = int(0.9*(len(data)))
train = data[:n]
test = data[n:]

In [12]:
# DataLoader Batch Size
batch_size = 4
# Maximum Context Length
block_size = 8

def get_batch(split):
    # Get Data
    inp = train if split == 'train' else test
    # Random Indexes
    ix = torch.randint(len(inp)-block_size, (batch_size,))
    # x is from i:i+block_size, y is i+1:i_block_size+1
    x = torch.stack([inp[i:i+block_size] for i in ix])
    y = torch.stack([inp[i+1:i+block_size+1] for i in ix])
    return x, y

x_batch, y_batch = get_batch('train')
for i in range(batch_size):
    for j in range(block_size):
        context = x_batch[i, :j+1]
        target = y_batch[i, j]
        print(f'Context is: {context.tolist()}, Target is {target.tolist()}')

Context is: [44], Target is 50
Context is: [44, 50], Target is 27
Context is: [44, 50, 27], Target is 44
Context is: [44, 50, 27, 44], Target is 34
Context is: [44, 50, 27, 44, 34], Target is 60
Context is: [44, 50, 27, 44, 34, 60], Target is 16
Context is: [44, 50, 27, 44, 34, 60, 16], Target is 38
Context is: [44, 50, 27, 44, 34, 60, 16, 38], Target is 54
Context is: [31], Target is 10
Context is: [31, 10], Target is 43
Context is: [31, 10, 43], Target is 19
Context is: [31, 10, 43, 19], Target is 24
Context is: [31, 10, 43, 19, 24], Target is 43
Context is: [31, 10, 43, 19, 24, 43], Target is 19
Context is: [31, 10, 43, 19, 24, 43, 19], Target is 24
Context is: [31, 10, 43, 19, 24, 43, 19, 24], Target is 23
Context is: [12], Target is 12
Context is: [12, 12], Target is 43
Context is: [12, 12, 43], Target is 56
Context is: [12, 12, 43, 56], Target is 36
Context is: [12, 12, 43, 56, 36], Target is 12
Context is: [12, 12, 43, 56, 36, 12], Target is 43
Context is: [12, 12, 43, 56, 36, 1

In [ ]:
# Seed
torch.maual_seed(1337)

# Bigram Model
class BigramLanguageModel(nn.Module):
